In [1]:
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from commit_transformer.tokenizer import build_vocab
from commit_transformer.preprosser import CommitDataset
from commit_transformer.model import CombinedModel

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
train = pd.read_csv('./datasets/PD_train.csv')
test = pd.read_csv('./datasets/PD_test.csv')
val = pd.read_csv('./datasets/PD_val.csv')

train.reset_index(inplace=True)
test.reset_index(inplace=True)
val.reset_index(inplace=True)
test.fillna('', inplace=True)
val.fillna('', inplace=True)
train.fillna('', inplace=True)

In [9]:
train

,index,Unnamed: 0,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
0,0,28208,,,1,ce9c22443e77594531be84ba8d523f4148ba09fe,commit ce9c22443e77594531be84ba8d523f4148ba09f...,diff --git a/drivers/media/v4l2-core/videobuf2...,torvalds,linux,wild
1,1,4003,,,0,8ad5db8a8ddbe3bd33078863a027011e28f1f4ee,commit 8ad5db8a8ddbe3bd33078863a027011e28f1f4e...,diff --git a/include/linux/kref.h b/include/li...,torvalds,linux,wild
2,2,10507,,,0,7bdc072086939093238a970f054e8e63d531253d,commit 7bdc072086939093238a970f054e8e63d531253...,diff --git a/drivers/gpu/drm/bridge/analogix/a...,torvalds,linux,wild
3,3,31853,CVE-2012-2794,CWE,1,5ad7335ebac2b38bb2a1c8df51a500b78461c05a,From 5ad7335ebac2b38bb2a1c8df51a500b78461c05a ...,diff --git a/libavcodec/indeo5.c b/libavcodec/...,,,cve
4,4,6625,,,0,2d1c4310774156204c3faff0db583337caeffcc4,commit 2d1c4310774156204c3faff0db583337caeffcc...,diff --git a/drivers/net/usb/cdc_ncm.c b/drive...,stoth68000,media-tree,wild
...,...,...,...,...,...,...,...,...,...,...,...
25065,25065,16850,,,0,28bb6da642cf2a419798b5825597809a1668aacb,commit 28bb6da642cf2a419798b5825597809a1668aac...,diff --git a/epan/dissectors/packet-iscsi.c b/...,wireshark,wireshark,wild
25066,25066,6265,,,0,b5fc571e4f730579f328ae9cf77435cb7fddc53d,commit b5fc571e4f730579f328ae9cf77435cb7fddc53...,diff --git a/libavcodec/aacdec.c b/libavcodec/...,FFmpeg,FFmpeg,wild
25067,25067,11284,,,0,cb4da1a34de3840ce49dc7292a063e1ef7f127af,commit cb4da1a34de3840ce49dc7292a063e1ef7f127a...,diff --git a/drivers/net/wireless/iwlwifi/iwl3...,stoth68000,media-tree,wild
25068,25068,860,,,0,fb8a0ddba29f91af5a2531b12c2e58bbdd69b844,commit fb8a0ddba29f91af5a2531b12c2e58bbdd69b84...,diff --git a/libmpdemux/demux_lavf.c b/libmpde...,mpv-player,mpv,wild


In [10]:
# df = df[['message','diff','label']] 
# df

In [11]:
train_data = []
for index, row in train.iterrows():
  train_data.append([row['commit_message'],row['diff_code'],row['category']])

val_data = []
for index, row in val.iterrows():
  val_data.append([row['commit_message'],row['diff_code'],row['category']])
test_data = []
for index, row in test.iterrows():
  test_data.append([row['commit_message'],row['diff_code'],row['category']])

In [12]:

# # Split the dataset
# train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# # Build vocabulary
vocab = build_vocab(train_data)

# Create Datasets and DataLoaders
train_dataset = CommitDataset(train_data, vocab)
val_dataset = CommitDataset(val_data, vocab)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [15]:


# Initialize the model
vocab_size = len(vocab)
embed_dim = 128  # Dimension for embeddings
num_heads = 8
hidden_dim = 512
num_layers = 6
dropout = 0.3

model = CombinedModel(vocab_size, embed_dim, num_heads, hidden_dim, num_layers, dropout)

# Train the model
model.trainer(train_loader, val_loader, num_epochs=10)

/root/miniconda3/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.6110397363753173
Epoch 2/10, Loss: 0.5349443248201496
Epoch 3/10, Loss: 0.48858871633554235
Epoch 4/10, Loss: 0.4705524073905373
Epoch 5/10, Loss: 0.4535194354437306
Epoch 6/10, Loss: 0.4414661721609908
Epoch 7/10, Loss: 0.4307894623019966
Epoch 8/10, Loss: 0.42141459845529483
Epoch 9/10, Loss: 0.4112089399911639
Epoch 10/10, Loss: 0.40164502816608777
Validation Accuracy: 0.7820584403498977
Precision: 0.6421471172962226
Recall: 0.7412507171543317
F1-Score: 0.6881491344873502


In [16]:
model.evaluate(val_loader)

Validation Accuracy: 0.7820584403498977
Precision: 0.6421471172962226
Recall: 0.7412507171543317
F1-Score: 0.6881491344873502
